Case 1) Test for exam clash 
module taken 
NA 

module intending to take
ACC 1701  
BSP1004
BSP1005 <- exam clash
BSP 1703 <-  exam clash (to only chose 1)
MKT1003
MNO1001
DSC1007

In [14]:
using ExcelReaders

csvdataprereq = readcsv("pre_req_df.csv")
headerprereq = csvdataprereq[1,:] #associated with columns
dataprereq = map(Int64,csvdataprereq[2:end,2:end])
modulecodes = csvdataprereq[2:end,1]
m = size(dataprereq, 1) #number of modules

#load module timeslots 

csvdatasched = readcsv("1718_s1_modsect_timeslots.csv")
headersched = csvdatasched[1,:]
datasched = csvdatasched[2:end,6:end] #stores a matrix: rows indicate modules 1 to n, columns indicate time Mon 8am to Fri 10pm

sectionalcodes = csvdatasched[2:end,5]

#csvmodstaken = readcsv("modules  taken v1.csv")
csvmodstaken = readxl("inputsheet v1.xlsm", "modules_taken!A1:B121")
modules_taken = map(Int64,csvmodstaken[:,2])

n = size(sectionalcodes, 1) #number of module-sectionals

t = size(datasched, 2) #number of timeslots altogether for 5 days
d = Int64(t/(t/5))

csvdatapreclu = readcsv("pre_clu_df.csv")
headerpreclu = csvdatapreclu[1,:]
datapreclu = map(Int64,csvdatapreclu[2:end,2:end])

csvconverter = readcsv("converter.csv")
headerconverter = csvconverter[1,:]
dataconverter = csvconverter[2:end,2:end] # m by n

#modules_wanted = readcsv("modules want v1.csv")
modules_wanted = readxl("inputsheet v1.xlsm", "modules_want!A1:B121")
datamodules_wanted = map(Int64,modules_wanted[1:end,2:end])


exam = readcsv("exam_sem1-edited.csv")
headerexam = exam[1,:]
dataexam = exam[2:end,2:end]

examdays = size(dataexam[:,:],2)

no_mod_prereq_satisf_vec = dataprereq*modules_taken
no_mod_prereq_needed_vec = sum(dataprereq',1)

using JuMP
using Cbc

model = Model(solver=CbcSolver())

#m = size(dataprereq[:,1], 1)
d = 5

#n = size(sectionalcodes, 1)#number of module-sectionals

#which modules should the student take
@variable(model, z[1:m], Bin)

#which sectionals should the student take
@variable(model, x[1:n], Bin)

#timetable decided by selection of x's
@variable(model, dx[1:t])

# Defining decision variables
#whether student needs to go to school on day i (assume that you change  from tday to t)
@variable(model, y[1:d], Bin)

# ensure that for each module only 1 sectional is taken
@constraint(model, Constraint1[j=1:m],sum(dataconverter[j,i]*x[i] for i = 1:n) == z[j])

# ensure that module is assigned only if he wants it
@constraint(model, Constraint2[j=1:m], z[j]<=datamodules_wanted[j])

# force workload to be 6 since can take all the mods except those that exam clashes
module_max = 6
module_min = 6
@constraint(model, sum(z[i] for i = 1:m )<=module_max)
@constraint(model, sum(z[i] for i = 1:m )>=module_min)

# ensure no timetable clashes
@constraint(model, dx .== datasched'*x)

@constraint(model, Constraint3[j=1:t], sum(datasched[i,j]*x[i] for i = 1:n) <= 1)


# ensure no exam clashes
@constraint(model, Constraint4[j=1:examdays], sum(dataexam[i,j]*x[i] for i = 1:m) <= 1)

# # of days going to school (need help)
b=1

for k=1:5 #5 days
    a = k*15
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=b:a)<=y[k]*15)
    b= b+15
end 

# lunch

a = 3 #10 - 11 am
b = 6 # 1 - 2 pm

for k=1:5 #5 days
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=a:b) <= 3)
    b = b + 15
    a = a + 15
end

# preclu constraints
@constraint(model,datapreclu*(z+modules_taken) .<= (1-(z+modules_taken))*m)

# prereq constraints
@constraint(model,no_mod_prereq_satisf_vec .>= no_mod_prereq_needed_vec'.*z)

# getting the rank of the preferences
using ExcelReaders
prefs = readxl("inputsheet v1.xlsm","preferences!A1:B3")

daysrank= abs(prefs[1,2]-4)
timecostrank = abs(prefs[2,2]-4)
utilrank = abs(prefs[3,2]-4)

# day cost
dayscost = fill(0.2, (1,d))*daysrank

# time cost
timecostfile = readcsv("timecost.csv")
timecost = map(float,timecostfile[:,2])*timecostrank

# sectional ranking 
utilfile = readxl("inputsheet v1.xlsm","sectionals_ranked_ratio!A1:B469")
utilvector = map(float,utilfile[:,2])*utilrank

#utilfile = readcsv("ranking calc v1.csv")


@objective(model, Max, sum(utilvector[i]*x[i] for i = 1:n)-sum(dayscost[i]*y[i] for i = 1:d)-sum(timecost[i]*dx[i] for i = 1:t))

solve(model)
# need to print out the answers on a csv
modulestotake = hcat(modulecodes,getvalue(z))
writecsv("modules to take v1", modulestotake)
timetable = hcat(headersched[6:end,:], getvalue(dx))
writecsv("timetable v1", timetable)
writecsv("sectionals to take v1.csv",  hcat(sectionalcodes, getvalue(x),utilvector))



Case 2) Preclusion 
module taken ACC1002

module intending to take
ACC 1701 <- preclude  
BSP1004
BSP1703 
MKT1003
MNO1001
DSC2008

In [19]:
using ExcelReaders

csvdataprereq = readcsv("pre_req_df.csv")
headerprereq = csvdataprereq[1,:] #associated with columns
dataprereq = map(Int64,csvdataprereq[2:end,2:end])
modulecodes = csvdataprereq[2:end,1]
m = size(dataprereq, 1) #number of modules

#load module timeslots 

csvdatasched = readcsv("1718_s1_modsect_timeslots.csv")
headersched = csvdatasched[1,:]
datasched = csvdatasched[2:end,6:end] #stores a matrix: rows indicate modules 1 to n, columns indicate time Mon 8am to Fri 10pm

sectionalcodes = csvdatasched[2:end,5]

#csvmodstaken = readcsv("modules  taken v2.csv")
csvmodstaken = readxl("inputsheet v2.xlsm", "modules_taken!A1:B121")
modules_taken = map(Int64,csvmodstaken[:,2])

n = size(sectionalcodes, 1) #number of module-sectionals

t = size(datasched, 2) #number of timeslots altogether for 5 days
d = Int64(t/(t/5))

csvdatapreclu = readcsv("pre_clu_df.csv")
headerpreclu = csvdatapreclu[1,:]
datapreclu = map(Int64,csvdatapreclu[2:end,2:end])

csvconverter = readcsv("converter.csv")
headerconverter = csvconverter[1,:]
dataconverter = csvconverter[2:end,2:end] # m by n

#modules_wanted = readcsv("modules want v2.csv")
modules_wanted = readxl("inputsheet v2.xlsm", "modules_want!A1:B121")
datamodules_wanted = map(Int64,modules_wanted[1:end,2:end])


exam = readcsv("exam_sem1-edited.csv")
headerexam = exam[1,:]
dataexam = exam[2:end,2:end]

examdays = size(dataexam[:,:],2)

no_mod_prereq_satisf_vec = dataprereq*modules_taken
no_mod_prereq_needed_vec = sum(dataprereq',1)

using JuMP
using Cbc

model = Model(solver=CbcSolver())

#m = size(dataprereq[:,1], 1)
d = 5

#n = size(sectionalcodes, 1)#number of module-sectionals

#which modules should the student take
@variable(model, z[1:m], Bin)

#which sectionals should the student take
@variable(model, x[1:n], Bin)

#timetable decided by selection of x's
@variable(model, dx[1:t])

# Defining decision variables
#whether student needs to go to school on day i (assume that you change  from tday to t)
@variable(model, y[1:d], Bin)

# ensure that for each module only 1 sectional is taken
@constraint(model, Constraint1[j=1:m],sum(dataconverter[j,i]*x[i] for i = 1:n) == z[j])

# ensure that module is assigned only if he wants it
@constraint(model, Constraint2[j=1:m], z[j]<=datamodules_wanted[j])

# force workload to be 6 since can take all the mods except those that exam clashes
module_max = 6
module_min = 5
@constraint(model, sum(z[i] for i = 1:m )<=module_max)
@constraint(model, sum(z[i] for i = 1:m )>=module_min)

# ensure no timetable clashes
@constraint(model, dx .== datasched'*x)

@constraint(model, Constraint3[j=1:t], sum(datasched[i,j]*x[i] for i = 1:n) <= 1)


# ensure no exam clashes
@constraint(model, Constraint4[j=1:examdays], sum(dataexam[i,j]*x[i] for i = 1:m) <= 1)

# # of days going to school (need help)
b=1

for k=1:5 #5 days
    a = k*15
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=b:a)<=y[k]*15)
    b= b+15
end 

# lunch

a = 3 #10 - 11 am
b = 6 # 1 - 2 pm

for k=1:5 #5 days
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=a:b) <= 3)
    b = b + 15
    a = a + 15
end

# preclu constraints
@constraint(model,datapreclu*(z+modules_taken) .<= (1-(z+modules_taken))*m)

# prereq constraints
@constraint(model,no_mod_prereq_satisf_vec .>= no_mod_prereq_needed_vec'.*z)

# getting the rank of the preferences
using ExcelReaders
prefs = readxl("inputsheet v2.xlsm","preferences!A1:B3")

daysrank= abs(prefs[1,2]-4)
timecostrank = abs(prefs[2,2]-4)
utilrank = abs(prefs[3,2]-4)

# day cost
dayscost = fill(0.2, (1,d))*daysrank

# time cost
timecostfile = readcsv("timecost.csv")
timecost = map(float,timecostfile[:,2])*timecostrank

# sectional ranking 
utilfile = readxl("inputsheet v2.xlsm","sectionals_ranked_ratio!A1:B469")
utilvector = map(float,utilfile[:,2])*utilrank

#utilfile = readcsv("ranking calc v2.csv")


@objective(model, Max, sum(utilvector[i]*x[i] for i = 1:n)-sum(dayscost[i]*y[i] for i = 1:d)-sum(timecost[i]*dx[i] for i = 1:t))

solve(model)
# need to print out the answers on a csv
modulestotake = hcat(modulecodes,getvalue(z))
writecsv("modules to take v2", modulestotake)
timetable = hcat(headersched[6:end,:], getvalue(dx))
writecsv("timetable v2", timetable)
writecsv("sectionals to take v2.csv",  hcat(sectionalcodes, getvalue(x),utilvector))



Case 3) PreReq
module taken ACC1002

Module to take
DSC3201 <- no pre-req  
BSP1004
BSP1703 
MKT1003
FIN2004
DSC2008

In [11]:
using ExcelReaders

csvdataprereq = readcsv("pre_req_df.csv")
headerprereq = csvdataprereq[1,:] #associated with columns
dataprereq = map(Int64,csvdataprereq[2:end,2:end])
modulecodes = csvdataprereq[2:end,1]
m = size(dataprereq, 1) #number of modules

#load module timeslots 

csvdatasched = readcsv("1718_s1_modsect_timeslots.csv")
headersched = csvdatasched[1,:]
datasched = csvdatasched[2:end,6:end] #stores a matrix: rows indicate modules 1 to n, columns indicate time Mon 8am to Fri 10pm

sectionalcodes = csvdatasched[2:end,5]

#csvmodstaken = readcsv("modules  taken v2.csv")
csvmodstaken = readxl("inputsheet v3.xlsm", "modules_taken!A1:B121")
modules_taken = map(Int64,csvmodstaken[:,2])

n = size(sectionalcodes, 1) #number of module-sectionals

t = size(datasched, 2) #number of timeslots altogether for 5 days
d = Int64(t/(t/5))

csvdatapreclu = readcsv("pre_clu_df.csv")
headerpreclu = csvdatapreclu[1,:]
datapreclu = map(Int64,csvdatapreclu[2:end,2:end])

csvconverter = readcsv("converter.csv")
headerconverter = csvconverter[1,:]
dataconverter = csvconverter[2:end,2:end] # m by n

#modules_wanted = readcsv("modules want v2.csv")
modules_wanted = readxl("inputsheet v3.xlsm", "modules_want!A1:B121")
datamodules_wanted = map(Int64,modules_wanted[1:end,2:end])


exam = readcsv("exam_sem1-edited.csv")
headerexam = exam[1,:]
dataexam = exam[2:end,2:end]

examdays = size(dataexam[:,:],2)

no_mod_prereq_satisf_vec = dataprereq*modules_taken
no_mod_prereq_needed_vec = sum(dataprereq',1)

using JuMP
using Cbc

model = Model(solver=CbcSolver())

#m = size(dataprereq[:,1], 1)
d = 5

#n = size(sectionalcodes, 1)#number of module-sectionals

#which modules should the student take
@variable(model, z[1:m], Bin)

#which sectionals should the student take
@variable(model, x[1:n], Bin)

#timetable decided by selection of x's
@variable(model, dx[1:t])

# Defining decision variables
#whether student needs to go to school on day i (assume that you change  from tday to t)
@variable(model, y[1:d], Bin)

# ensure that for each module only 1 sectional is taken
@constraint(model, Constraint1[j=1:m],sum(dataconverter[j,i]*x[i] for i = 1:n) == z[j])

# ensure that module is assigned only if he wants it
@constraint(model, Constraint2[j=1:m], z[j]<=datamodules_wanted[j])

# force workload to be 6 since can take all the mods except those that exam clashes
module_max = 6
module_min = 5
@constraint(model, sum(z[i] for i = 1:m )<=module_max)
@constraint(model, sum(z[i] for i = 1:m )>=module_min)

# ensure no timetable clashes
@constraint(model, dx .== datasched'*x)

@constraint(model, Constraint3[j=1:t], sum(datasched[i,j]*x[i] for i = 1:n) <= 1)


# ensure no exam clashes
@constraint(model, Constraint4[j=1:examdays], sum(dataexam[i,j]*x[i] for i = 1:m) <= 1)

# # of days going to school (need help)
b=1

for k=1:5 #5 days
    a = k*15
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=b:a)<=y[k]*15)
    b= b+15
end 

# lunch

a = 3 #10 - 11 am
b = 6 # 1 - 2 pm

for k=1:5 #5 days
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=a:b) <= 3)
    b = b + 15
    a = a + 15
end

# preclu constraints
@constraint(model,datapreclu*(z+modules_taken) .<= (1-(z+modules_taken))*m)

# prereq constraints
@constraint(model,no_mod_prereq_satisf_vec .>= no_mod_prereq_needed_vec'.*z)

# getting the rank of the preferences
using ExcelReaders
prefs = readxl("inputsheet v3.xlsm","preferences!A1:B3")

daysrank= abs(prefs[1,2]-4)
timecostrank = abs(prefs[2,2]-4)
utilrank = abs(prefs[3,2]-4)

# day cost
dayscost = fill(0.2, (1,d))*daysrank

# time cost
timecostfile = readcsv("timecost.csv")
timecost = map(float,timecostfile[:,2])*timecostrank

# sectional ranking 
utilfile = readxl("inputsheet v3.xlsm","sectionals_ranked_ratio!A1:B469")
utilvector = map(float,utilfile[:,2])*utilrank

#utilfile = readcsv("ranking calc v2.csv")


@objective(model, Max, sum(utilvector[i]*x[i] for i = 1:n)-sum(dayscost[i]*y[i] for i = 1:d)-sum(timecost[i]*dx[i] for i = 1:t))

solve(model)
# need to print out the answers on a csv
modulestotake = hcat(modulecodes,getvalue(z))
writecsv("modules to take v3", modulestotake)
timetable = hcat(headersched[6:end,:], getvalue(dx))
writecsv("timetable v3", timetable)
writecsv("sectionals to take v3",  hcat(sectionalcodes, getvalue(x),utilvector))



Mods to test:
    FIN3101A - No Prereq
    BSP1005
    DSC1007
    FIN2004
    ACC1701 - Preclu
    ACC1006
    BSP1004   

In [9]:
using ExcelReaders

csvdataprereq = readcsv("pre_req_df.csv")
headerprereq = csvdataprereq[1,:] #associated with columns
dataprereq = map(Int64,csvdataprereq[2:end,2:end])
modulecodes = csvdataprereq[2:end,1]
m = size(dataprereq, 1) #number of modules

#load module timeslots 

csvdatasched = readcsv("1718_s1_modsect_timeslots.csv")
headersched = csvdatasched[1,:]
datasched = csvdatasched[2:end,6:end] #stores a matrix: rows indicate modules 1 to n, columns indicate time Mon 8am to Fri 10pm

sectionalcodes = csvdatasched[2:end,5]

#csvmodstaken = readcsv("modules  taken v2.csv")
csvmodstaken = readxl("inputsheet v4.xlsm", "modules_taken!A1:B121")
modules_taken = map(Int64,csvmodstaken[:,2])

n = size(sectionalcodes, 1) #number of module-sectionals

t = size(datasched, 2) #number of timeslots altogether for 5 days
d = Int64(t/(t/5))

csvdatapreclu = readcsv("pre_clu_df.csv")
headerpreclu = csvdatapreclu[1,:]
datapreclu = map(Int64,csvdatapreclu[2:end,2:end])

csvconverter = readcsv("converter.csv")
headerconverter = csvconverter[1,:]
dataconverter = csvconverter[2:end,2:end] # m by n

#modules_wanted = readcsv("modules want v2.csv")
modules_wanted = readxl("inputsheet v4.xlsm", "modules_want!A1:B121")
datamodules_wanted = map(Int64,modules_wanted[1:end,2:end])


exam = readcsv("exam_sem1-edited.csv")
headerexam = exam[1,:]
dataexam = exam[2:end,2:end]

examdays = size(dataexam[:,:],2)

no_mod_prereq_satisf_vec = dataprereq*modules_taken
no_mod_prereq_needed_vec = sum(dataprereq',1)

using JuMP
using Cbc

model = Model(solver=CbcSolver())

#m = size(dataprereq[:,1], 1)
d = 5

#n = size(sectionalcodes, 1)#number of module-sectionals

#which modules should the student take
@variable(model, z[1:m], Bin)

#which sectionals should the student take
@variable(model, x[1:n], Bin)

#timetable decided by selection of x's
@variable(model, dx[1:t])

# Defining decision variables
#whether student needs to go to school on day i (assume that you change  from tday to t)
@variable(model, y[1:d], Bin)

# ensure that for each module only 1 sectional is taken
@constraint(model, Constraint1[j=1:m],sum(dataconverter[j,i]*x[i] for i = 1:n) == z[j])

# ensure that module is assigned only if he wants it
@constraint(model, Constraint2[j=1:m], z[j]<=datamodules_wanted[j])

# force workload to be 6 since can take all the mods except those that exam clashes
module_max = 6
module_min = 5
@constraint(model, sum(z[i] for i = 1:m )<=module_max)
@constraint(model, sum(z[i] for i = 1:m )>=module_min)

# ensure no timetable clashes
@constraint(model, dx .== datasched'*x)

@constraint(model, Constraint3[j=1:t], sum(datasched[i,j]*x[i] for i = 1:n) <= 1)


# ensure no exam clashes
@constraint(model, Constraint4[j=1:examdays], sum(dataexam[i,j]*x[i] for i = 1:m) <= 1)

# # of days going to school (need help)
b=1

for k=1:5 #5 days
    a = k*15
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=b:a)<=y[k]*15)
    b= b+15
end 

# lunch

a = 3 #10 - 11 am
b = 6 # 1 - 2 pm

for k=1:5 #5 days
    @constraint(model,sum(datasched[i,j]*x[i] for i = 1:n, j=a:b) <= 3)
    b = b + 15
    a = a + 15
end

# preclu constraints
@constraint(model,datapreclu*(z+modules_taken) .<= (1-(z+modules_taken))*m)

# prereq constraints
@constraint(model,no_mod_prereq_satisf_vec .>= no_mod_prereq_needed_vec'.*z)

# getting the rank of the preferences
using ExcelReaders
prefs = readxl("inputsheet v4.xlsm","preferences!A1:B3")

daysrank= abs(prefs[1,2]-4)
timecostrank = abs(prefs[2,2]-4)
utilrank = abs(prefs[3,2]-4)

# day cost
dayscost = fill(0.2, (1,d))*daysrank

# time cost
timecostfile = readcsv("timecost.csv")
timecost = map(float,timecostfile[:,2])*timecostrank

# sectional ranking 
utilfile = readxl("inputsheet v4.xlsm","sectionals_ranked_ratio!A1:B469")
utilvector = map(float,utilfile[:,2])*utilrank

#utilfile = readcsv("ranking calc v2.csv")


@objective(model, Max, sum(utilvector[i]*x[i] for i = 1:n)-sum(dayscost[i]*y[i] for i = 1:d)-sum(timecost[i]*dx[i] for i = 1:t))

solve(model)
# need to print out the answers on a csv
modulestotake = hcat(modulecodes,getvalue(z))
writecsv("modules to take v4", modulestotake)
timetable = hcat(headersched[6:end,:], getvalue(dx))
writecsv("timetable v4", timetable)
writecsv("sectionals to take v4",  hcat(sectionalcodes, getvalue(x),utilvector))

